In [1]:
import numpy as np
from sega_learn.neural_networks import *

In [2]:
# Sample data
sample_size = 100_000

# Generate random data
np.random.seed(0)
X_train = np.random.randn(sample_size, 10)
y_train = np.random.randint(0, 2, sample_size)

X_val = np.random.randn(20, 10)
y_val = np.random.randint(0, 2, 20)

In [3]:
# Neural network configurations
layer_sizes = [10, 5, 3, 2]
dropout_rate = 0
reg_lambda = 0
activations = ["relu", "relu", "softmax"]

In [4]:
# Initialize neural networks
nn_numba = NumbaBackendNeuralNetwork(
    layer_sizes, dropout_rate, reg_lambda, activations, compile_numba=False
)
nn_no_numba = BaseBackendNeuralNetwork(
    layer_sizes, dropout_rate, reg_lambda, activations
)

In [5]:
# Models are initialized with the same parameters but receive different randomized weights and biases
def set_weights_biases(nn_numba, nn_no_numba):
    # Set weights to the same values for comparison
    for i in range(len(nn_numba.weights)):
        nn_numba.weights[i] = np.random.randn(*nn_numba.weights[i].shape)
        nn_no_numba.weights[i] = nn_numba.weights[i].copy()

    # Set biases to the same values for comparison
    for i in range(len(nn_numba.biases)):
        nn_numba.biases[i] = np.random.randn(*nn_numba.biases[i].shape)
        nn_no_numba.biases[i] = nn_numba.biases[i].copy()


set_weights_biases(nn_numba, nn_no_numba)

# Assert equality of weights
for w1, w2 in zip(nn_numba.weights, nn_no_numba.weights, strict=False):
    assert np.array_equal(w1, w2), "Weights are not equal!"
print("Weights are equal!")

# Assert equality of biases
for b1, b2 in zip(nn_numba.biases, nn_no_numba.biases, strict=False):
    assert np.array_equal(b1, b2), "Biases are not equal!"
print("Biases are equal!")

Weights are equal!
Biases are equal!


In [6]:
# Model Layers are initialized with the same parameters but receive different randomized weights and biases

# Set weights and biases to the same values for comparison


def set_layer_weights_biases(nn_numba, nn_no_numba):
    for i in range(len(nn_numba.layers)):
        nn_numba.layers[i].weights = np.random.randn(*nn_numba.layers[i].weights.shape)
        nn_no_numba.layers[i].weights = nn_numba.layers[i].weights.copy()

        nn_numba.layers[i].biases = np.random.randn(*nn_numba.layers[i].biases.shape)
        nn_no_numba.layers[i].biases = nn_numba.layers[i].biases.copy()


set_layer_weights_biases(nn_numba, nn_no_numba)

# Assert equality of weights
for l1, l2 in zip(nn_numba.layers, nn_no_numba.layers, strict=False):
    assert np.array_equal(l1.weights, l2.weights), "Layer weights are not equal!"
print("Layer weights are equal!")

# Assert equality of biases
for l1, l2 in zip(nn_numba.layers, nn_no_numba.layers, strict=False):
    assert np.array_equal(l1.biases, l2.biases), "Layer biases are not equal!"
print("Layer biases are equal!")

Layer weights are equal!
Layer biases are equal!


In [7]:
# Forward pass comparison
output_numba = nn_numba.forward(X_train)
output_no_numba = nn_no_numba.forward(X_train)

tolerance = 1e-7
if np.allclose(output_numba, output_no_numba, atol=tolerance):
    print(f"Forward pass outputs are equal within tolerance of {tolerance}.")
else:
    print(f"Forward pass outputs are not equal within tolerance of {tolerance}.")
    print(f"First 5 outputs (Numba): \n{output_numba[:5]}")
    print(f"First 5 outputs (No Numba): \n{output_no_numba[:5]}")

Forward pass outputs are equal within tolerance of 1e-07.


In [8]:
# Backward pass comparison
nn_numba.backward(y_train)
nn_no_numba.backward(y_train)

# Layers gradients comparison
# Compare gradients

tolerance = 1e-7

print("\nComparing gradients:")
print("-" * 75, end="")
for i in range(len(nn_numba.layers)):
    weights_close = np.allclose(
        nn_numba.layers[i].weight_gradients,
        nn_no_numba.layers[i].weight_gradients,
        atol=tolerance,
    )
    if not weights_close:
        print(
            f"\nLayer {i} weight gradients are not equal within tolerance of {tolerance}."
        )
        # Find index of first non-equal element
        diff_index = np.where(
            nn_numba.layers[i].weight_gradients
            != nn_no_numba.layers[i].weight_gradients
        )[0][0]
        print(f"First non-equal element at index {diff_index}:")
        print(
            f"Numba   {diff_index}: {nn_numba.layers[i].weight_gradients[diff_index]}"
        )
        print(
            f"Vanilla {diff_index}: {nn_no_numba.layers[i].weight_gradients[diff_index]}"
        )

    else:
        print(
            f"\nLayer {i} weight gradients are equal within tolerance of {tolerance}."
        )

print("\nComparing biases:")
print("-" * 75, end="")
for i in range(len(nn_numba.layers)):
    biases_close = np.allclose(
        nn_numba.layers[i].bias_gradients,
        nn_no_numba.layers[i].bias_gradients,
        atol=tolerance,
    )
    if not biases_close:
        print(
            f"\nLayer {i} bias gradients are not equal within tolerance of {tolerance}."
        )
        # Find index of first non-equal element
        diff_index = np.where(
            nn_numba.layers[i].bias_gradients != nn_no_numba.layers[i].bias_gradients
        )[0][0]
        print(f"First non-equal element at index {diff_index}:")
        print(f"Numba   {diff_index}: {nn_numba.layers[i].bias_gradients[diff_index]}")
        print(
            f"Vanilla {diff_index}: {nn_no_numba.layers[i].bias_gradients[diff_index]}"
        )

    else:
        print(f"\nLayer {i} bias gradients are equal within tolerance of {tolerance}.")


Comparing gradients:
---------------------------------------------------------------------------
Layer 0 weight gradients are equal within tolerance of 1e-07.

Layer 1 weight gradients are equal within tolerance of 1e-07.

Layer 2 weight gradients are equal within tolerance of 1e-07.

Comparing biases:
---------------------------------------------------------------------------
Layer 0 bias gradients are equal within tolerance of 1e-07.

Layer 1 bias gradients are equal within tolerance of 1e-07.

Layer 2 bias gradients are equal within tolerance of 1e-07.


In [9]:
# Initialize neural networks
nn_numba = NumbaBackendNeuralNetwork(
    layer_sizes, dropout_rate, reg_lambda, activations, compile_numba=False
)
nn_no_numba = BaseBackendNeuralNetwork(
    layer_sizes, dropout_rate, reg_lambda, activations
)

set_weights_biases(nn_numba, nn_no_numba)
set_layer_weights_biases(nn_numba, nn_no_numba)

# Evaluation comparison
accuracy_numba, _ = nn_numba.evaluate(X_val, y_val)
accuracy_no_numba, _ = nn_no_numba.evaluate(X_val, y_val)

tolerance = 1e-2
print(
    f"Evaluation accuracies are equal to within tolerance of {tolerance}: {np.allclose(accuracy_numba, accuracy_no_numba, atol=tolerance)}"
)
print(f"\tNumba accuracy:   {accuracy_numba}\n\tVanilla accuracy: {accuracy_no_numba}")

Evaluation accuracies are equal to within tolerance of 0.01: True
	Numba accuracy:   0.65
	Vanilla accuracy: 0.65
